In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('Data_prep.csv', low_memory = False)

In [3]:
col_desc = pd.read_csv('Column_dictionary.csv', low_memory = False)

In [4]:
data.head()

,communityname,state,countyCode,communityCode,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
0,BerkeleyHeightstownship,NJ,39,5320,1,11980,3.10,1.37,91.78,6.50,...,14,114.85,138,1132.08,16,131.26,2,16.41,41.02,1394.59
1,Marpletownship,PA,45,47616,1,23123,2.82,0.80,95.57,3.44,...,57,242.37,376,1598.78,26,110.55,1,4.25,127.56,1955.95
2,Tigardcity,OR,?,?,1,29344,2.43,0.74,94.33,3.43,...,274,758.14,1797,4972.19,136,376.3,22,60.87,218.59,6167.51
3,Gloversvillecity,NY,35,29443,1,16656,2.40,1.70,97.35,0.50,...,225,1301.78,716,4142.56,47,271.93,?,?,306.64,?
4,Bemidjicity,MN,7,5068,1,11245,2.76,0.53,89.16,1.17,...,91,728.93,1060,8490.87,91,728.93,5,40.05,?,9988.79


In [5]:
col_desc_list = col_desc.columns.tolist()

In [6]:
# split values so the dictionary can be created
my_list = []

for val in col_desc_list:
    my_list.append(val.split(':'))

In [7]:
my_list_keys = []
my_list_val = []

for val in my_list:
    my_list_keys.append(val[0])
    my_list_val.append(val[1])

In [8]:
column_dictionary = dict(zip(my_list_keys, my_list_val))

In [9]:
column_dictionary

{'communityname': ' Community name - not predictive - for information only (string)',
 'state': ' US state (by 2 letter postal abbreviation)(nominal)',
 'countyCode': ' numeric code for county - not predictive, and many missing values (numeric)',
 'communityCode': ' numeric code for community - not predictive and many missing values (numeric)',
 'fold': ' fold number for non-random 1 fold cross validation, potentially useful for debugging, paired tests - not predictive (numeric - integer)',
 'population': ' population for community',
 'householdsize': ' mean people per household (numeric - decimal)',
 'racepctblack': ' percentage of population that is african american (numeric - decimal)',
 'racePctWhite': ' percentage of population that is caucasian (numeric - decimal)',
 'racePctAsian': ' percentage of population that is of asian heritage (numeric - decimal)',
 'racePctHisp': ' percentage of population that is of hispanic heritage (numeric - decimal)',
 'agePct12t21': ' percentage of

In [10]:
# 6. dokonać imputacji braków, wybraną metodą, dla przynajmniej 4 różnych zmiennych

In [11]:
# check how many zeroes per column and not more than 6 missing values
imp_columns = []

for i in range (len(data.isin([0]).sum())):
    
    if data.isin([0]).sum()[i] != 0 and data.isin([0]).sum()[i] < 6:
        
        imp_columns.append(data.isin([0]).sum().index[i]) # will use it later
        
        print(data.isin([0]).sum().index[i], data.isin([0]).sum()[i])

racepctblack 2
NumKidsBornNeverMar 5
PctKidsBornNeverMar 5
OwnOccQrange 5
RentQrange 2


# Replacing missing values with the mean value

In [12]:
data.describe()

,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,...,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,murders,murdPerPop
count,2215.000000,2.215000e+03,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,...,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000
mean,5.494357,5.311798e+04,2.707327,9.335102,83.979819,2.670203,7.950176,14.445837,27.644840,13.975142,...,61.539630,51.538596,77.411079,88.111865,27.419955,2783.835034,3.041124,0.980163,7.764786,5.859296
std,2.872924,2.046203e+05,0.334120,14.247156,16.419080,4.473843,14.589832,4.518623,6.181517,5.970747,...,16.750061,10.517926,10.878186,7.287836,109.822600,2828.993341,4.912917,2.877128,58.166468,9.156829
min,1.000000,1.000500e+04,1.600000,0.000000,2.680000,0.030000,0.120000,4.580000,9.380000,4.640000,...,6.750000,11.830000,27.950000,32.830000,0.900000,10.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,1.436600e+04,2.500000,0.860000,76.320000,0.620000,0.930000,12.250000,24.415000,11.320000,...,50.110000,44.995000,72.060000,85.200000,7.300000,1181.900000,0.360000,0.000000,0.000000,0.000000
50%,5.000000,2.279200e+04,2.660000,2.870000,90.350000,1.230000,2.180000,13.620000,26.780000,12.540000,...,64.490000,52.170000,79.490000,90.030000,13.700000,2027.300000,1.220000,0.000000,1.000000,2.170000
75%,8.000000,4.302400e+04,2.850000,11.145000,96.225000,2.670000,7.810000,15.360000,29.205000,14.345000,...,74.855000,58.740000,85.135000,93.010000,26.100000,3321.700000,3.365000,0.000000,3.000000,8.365000
max,10.000000,7.322564e+06,5.280000,96.670000,99.630000,57.460000,95.290000,54.400000,70.510000,63.620000,...,93.140000,78.560000,96.590000,99.900000,3569.800000,44229.900000,54.330000,48.440000,1946.000000,91.090000


In [13]:
data.describe().RentQrange[1]

197.286230248307

In [12]:
# rows with values to be replaced
data[data['racepctblack']== 0]

,communityname,state,countyCode,communityCode,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
594,Vidorcity,TX,?,?,3,10935,2.78,0.0,99.17,0.16,...,75,648.28,391,3379.72,28,242.03,3,25.93,267.96,4295.96
1820,Martinsvillecity,IN,?,?,9,11677,2.65,0.0,99.43,0.15,...,59,473.9,408,3277.11,20,160.64,3,24.1,216.87,3935.74


In [14]:
imp_columns

['racepctblack',
 'NumKidsBornNeverMar',
 'PctKidsBornNeverMar',
 'OwnOccQrange',
 'RentQrange']

In [15]:
data['racepctblack'][594]

0.0

In [16]:
# not a nice way to do it but it works!
#data['racepctblack'][594] = data.describe().racepctblack[1]
#data['racepctblack'][1820] = data.describe().racepctblack[1]

#data['NumKidsBornNeverMar'][625] = data.describe().NumKidsBornNeverMar[1]
#data['NumKidsBornNeverMar'][862] = data.describe().NumKidsBornNeverMar[1]
#data['NumKidsBornNeverMar'][1479] = data.describe().NumKidsBornNeverMar[1]
#data['NumKidsBornNeverMar'][1735] = data.describe().NumKidsBornNeverMar[1]
#data['NumKidsBornNeverMar'][1965] = data.describe().NumKidsBornNeverMar[1]

#data['PctKidsBornNeverMar'][625] = data.describe().PctKidsBornNeverMar[1]
#data['PctKidsBornNeverMar'][862] = data.describe().PctKidsBornNeverMar[1]
#data['PctKidsBornNeverMar'][1479] = data.describe().PctKidsBornNeverMar[1]
#data['PctKidsBornNeverMar'][1735] = data.describe().PctKidsBornNeverMar[1]
#data['PctKidsBornNeverMar'][1965] = data.describe().PctKidsBornNeverMar[1]

#data['RentQrange'][597] = data.describe().RentQrange[1]
#data['RentQrange'][2105] = data.describe().RentQrange[1]

In [31]:
# nicer way to do the above
for col in imp_columns:
    
    for row in data[data[col]== 0].index:
        
        data[col][row] = data.describe()[col][1]

/Users/imac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [196]:
# 8. dokonać normalizacji zbioru danych przynajmniej 4 wybranymi metodami
# przedstawić wybrane metody
# uzasadnić wybór metody
# do dalszej analizy wybrać jeden wariant

In [27]:
data.describe()

,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,...,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,murders,murdPerPop
count,2215.000000,2.215000e+03,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,...,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000
mean,5.494357,5.311798e+04,2.707327,9.335102,83.979819,2.670203,7.950176,14.445837,27.644840,13.975142,...,61.539630,51.538596,77.411079,88.111865,27.419955,2783.835034,3.041124,0.980163,7.764786,5.859296
std,2.872924,2.046203e+05,0.334120,14.247156,16.419080,4.473843,14.589832,4.518623,6.181517,5.970747,...,16.750061,10.517926,10.878186,7.287836,109.822600,2828.993341,4.912917,2.877128,58.166468,9.156829
min,1.000000,1.000500e+04,1.600000,0.000000,2.680000,0.030000,0.120000,4.580000,9.380000,4.640000,...,6.750000,11.830000,27.950000,32.830000,0.900000,10.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,1.436600e+04,2.500000,0.860000,76.320000,0.620000,0.930000,12.250000,24.415000,11.320000,...,50.110000,44.995000,72.060000,85.200000,7.300000,1181.900000,0.360000,0.000000,0.000000,0.000000
50%,5.000000,2.279200e+04,2.660000,2.870000,90.350000,1.230000,2.180000,13.620000,26.780000,12.540000,...,64.490000,52.170000,79.490000,90.030000,13.700000,2027.300000,1.220000,0.000000,1.000000,2.170000
75%,8.000000,4.302400e+04,2.850000,11.145000,96.225000,2.670000,7.810000,15.360000,29.205000,14.345000,...,74.855000,58.740000,85.135000,93.010000,26.100000,3321.700000,3.365000,0.000000,3.000000,8.365000
max,10.000000,7.322564e+06,5.280000,96.670000,99.630000,57.460000,95.290000,54.400000,70.510000,63.620000,...,93.140000,78.560000,96.590000,99.900000,3569.800000,44229.900000,54.330000,48.440000,1946.000000,91.090000
